In [4]:
# read, parse and serialize graph to nt format
import time
import rdflib
from rdflib.graph import Graph, ConjunctiveGraph
import rdflib.namespace
from rdflib.namespace import OWL, RDF, RDFS
from rdflib import Namespace, URIRef, Literal

print ('STARTING')
start = time.time()

s = '../../../../Volumes/Public/SeminarPaper/example1.ttl'
g = rdflib.ConjunctiveGraph()
print ('START PARSING GRAPH')
g = g.parse(s, format="n3")
print ('DONE PARSING GRAPH')
print ('START SERIALIZING GRAPH')
#g.serialize(destination='Wikidata/test-30.nt', format='nt')
print g.serialize(format='nt')
print ('DONE SERIALIZING GRAPH')
print ('EXECUTION TIME: {} s'.format(time.time()-start))

STARTING
START PARSING GRAPH
DONE PARSING GRAPH
START SERIALIZING GRAPH
_:ub3bL6C13 <http://example.org/stuff/1.0/fullname> "Dave Beckett" .
<http://www.w3.org/TR/rdf-syntax-grammar> <http://purl.org/dc/elements/1.1/title> "RDF/XML Syntax Specification (Revised)" .
_:ub3bL6C13 <http://example.org/stuff/1.0/homePage> <http://purl.org/net/dajobe/> .
<http://www.w3.org/TR/rdf-syntax-grammar> <http://example.org/stuff/1.0/editor> _:ub3bL6C13 .


DONE SERIALIZING GRAPH
EXECUTION TIME: 0.020387172699 s


In [2]:
# get file lines
import time

file = 'Wikidata/wikidata-20160801-all-BETA.ttl'
start = time.time()
print('START')

# from http://stackoverflow.com/questions/845058/how-to-get-line-count-cheaply-in-python
num_lines = sum(1 for line in open(file))
print ('{} has {} lines'.format(file, num_lines))
print ('EXECUTION TIME: {} s'.format(time.time()-start))

START
Wikidata/wikidata-20160801-all-BETA.ttl has 1933695292 lines
EXECUTION TIME: 403.305052042 s


In [ ]:
# get remove invalid escape `\a' at line 544253967
import time

file = '/Volumes/Samsung/wikidata-20160801-all-BETA.ttl'
start = time.time()
print('START')
counter = 0
#with open(file) as f:
#    for i, line in enumerate(f):
#        if (i > 544253960):
#            print i
#            print line
#            if '\a' in line:
#                print ('found \a')
#        if (i > 544253970):
#            break            
    
print ('EXECUTION TIME: {} s'.format(time.time()-start))
#
#output
#544253966
#	wdt:P487 "\a" ;


START
EXECUTION TIME: 0.00662589073181 s


In [1]:
# delete lines with escape character \a
import time

oldfile = '/Volumes/Samsung/wikidata-20160801-all-BETA.ttl'
newFile = 'Wikidata/wikidata-20160801-all-BETA_v2.ttl'

deletedLines = set()
lineCounter = 0
start = time.time()
print('START')

#f = open(oldfile, 'r')
#fn = open(newFile, 'w')
#lines = f.readlines()
#for line in lines:
#    if '\a' not in line:
#        fn.write(line)
#    else:
#        deletedLines.add(line)
#        print ('line deleted')
#    lineCounter += 1
#    if (lineCounter % 100000000 == 0):
#        print ('{} lines read'.format(lineCounter))
#f.close()
#fn.close()

print ('{} lines found containing \a'.format(len(deletedLines)))
print deletedLines
print ('EXECUTION TIME: {} s'.format(time.time()-start))


START
0 lines found containing 
set([])
EXECUTION TIME: 0.000699996948242 s


In [2]:
# create sample
import os
import string
import itertools
import time

#sampleFile = '../../../../Volumes/Public/SeminarPaper/wikidata-20160801-all-BETA_sample.ttl'
#sampleFile = 'Wikidata/wikidata-20160801-all-BETA_sample_1m.ttl'
sampleFile = '/Volumes/Samsung/wikidata_sample_from_nt.nt'
#readFile = '../../../../Volumes/Public/SeminarPaper/wikidata-20160801-all-BETA.ttl'
#readFile = 'Wikidata/wikidata-20160801-all-BETA.ttl'
readFile = '/Volumes/Samsung//wikidata.nt'


sampleSize = 1000000
start = time.time()
print('START')
try:
    # read file
    f = open(readFile, 'r')
    # write file
    fw = open(sampleFile, 'w')
    lineCounter = 0
    for line in f:
        if (lineCounter < sampleSize):
            fw.write(line)
            lineCounter += 1
        else:
            print ('{} lines processed'.format(sampleSize))
            break
    f.close()
    print ('DONE')
except:
    print('ERROR')
print ('EXECUTION TIME: {} s'.format(time.time()-start))

START
1000000 lines processed
DONE
EXECUTION TIME: 6.44111514091 s


In [4]:
# get top 10 classes

import os
import time
import string
import itertools
import re
import operator

start = time.time()

readFile = 'Wikidata/wikidata_sample_1m.nt'
#readFile = '../../../../Volumes/Public/SeminarPaper/wikidata-20160801-all-BETA.ttl'

classCount = {} #key:class, value:count

lineProgress = 1000000

# ADAPT TO WIKIDATA
rdfType = '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>'
owlClass = '<http://www.w3.org/2002/07/owl#Class>'

# <http://www.wikidata.org/prop/novalue/P16> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Class> .


allClasses = set()
seenClasses = set()
classCountDic = {} #key:class, value:count

def getSPO(splittedLine):
    word_position = 0
    for word in splittedLine:
        if (word_position == 0):
            subj = word
        elif (word_position == 1):
            pred = word
        elif (word_position == 2):
            obj = word
        else:
            return subj, pred, obj
        word_position += 1
    return subj, pred, obj

def addToClasses(s):
    if s not in allClasses:
        allClasses.add(s)
        
def countClasses(o):
    #print ('countClasses')
    if o in seenClasses:
        classCountDic[o] += 1
    else:
        classCountDic[o] = 1
        seenClasses.add(o)
        
print('START')
try:
    f = open(readFile, 'r')
    lineCounter = 0
    for line in f:
        splittedLine = line.rstrip('\n').split()
        s, p, o = getSPO(splittedLine)
        if (p == rdfType and o == owlClass):
            #print ('{} {} {}'.format(s,p,o))
            addToClasses(s)
        lineCounter += 1
        if (lineCounter % lineProgress == 0):
            print ('{} lines read'.format(lineCounter))
        #if (lineCounter > 100):
        #    break
    f.close()
    print ('{} classes found'.format(len(allClasses)))
    f = open(readFile, 'r')
    lineCounter = 0
    for line in f:
        splittedLine = line.rstrip('\n').split()
        s, p, o = getSPO(splittedLine)
        if (p == rdfType and o in allClasses):
            #print ('{}'.format(o))
            countClasses(o)
        lineCounter += 1
        if (lineCounter % lineProgress == 0):
            print ('{} lines read'.format(lineCounter))
    #print allClasses
    #print classCountDic
    top10dic = dict(sorted(classCountDic.items(), key=operator.itemgetter(1), reverse=True)[:10])
    print (sorted(top10dic.items(), key=operator.itemgetter(1), reverse=True))
    f.close()
    print ('DONE')
except:
    print('ERROR')
print ('EXECUTION TIME: {} s'.format(time.time()-start))

START
598 classes found
[('<http://www.wikidata.org/prop/novalue/P37>', 8), ('<http://www.wikidata.org/prop/novalue/P102>', 4), ('<http://www.wikidata.org/prop/novalue/P155>', 1), ('<http://www.wikidata.org/prop/novalue/P2555>', 1)]
DONE
EXECUTION TIME: 7.11636400223 s


In [2]:
# calculate class indegree & outdegree

import operator
import time

print ('STARTING')
start = time.time()

readFile = '/Volumes/Samsung/wikidata_sample_from_nt.nt'
#readFile = '/Volumes/Samsung/wikidata.nt'

indegreeDic = {} # key:class, value: indegree
outdegreeDic = {} #key: class, value: outdegree

lineProgress = 10000000

top10set = set()
top10set.update(['<http://www.wikidata.org/entity/Q5>', '<http://www.wikidata.org/entity/Q4167836>', '<http://www.wikidata.org/entity/Q16521>', '<http://www.wikidata.org/entity/Q4167410>', '<http://www.wikidata.org/entity/Q11266439>', '<http://www.wikidata.org/entity/Q13100073>', '<http://www.wikidata.org/entity/Q486972>', '<http://www.wikidata.org/entity/Q532>', '<http://www.wikidata.org/entity/Q79007>', '<http://www.wikidata.org/entity/Q13406463>'])

def getSPO(splittedLine):
    word_position = 0
    for word in splittedLine:
        if (word_position == 0):
            subj = word
        elif (word_position == 1):
            pred = word
        elif (word_position == 2):
            obj = word
        else:
            return subj, pred, obj
        word_position += 1

try:
    f = open(readFile, 'r')
    lineCounter = 0
    for line in f:
        splittedLine = line.rstrip('\n').split()
        s, p, o = getSPO(splittedLine)
        if s in top10set:
            if outdegreeDic.has_key(s):
                outdegreeDic[s] += 1
            else:
                outdegreeDic[s] = 1
        if o in top10set:
            if indegreeDic.has_key(o):
                indegreeDic[o] += 1
            else:
                indegreeDic[o] = 1
        lineCounter += 1
        if (lineCounter % lineProgress == 0):
            print ('{} lines read'.format(lineCounter))
    f.close()
    print ('DONE')
    print ('INDEGREE RESULT:')
    print sorted(indegreeDic.items(), key=operator.itemgetter(1), reverse=True)
    print ('OUTDEGREE RESULT')
    print sorted(outdegreeDic.items(), key=operator.itemgetter(1), reverse=True)
    print ('EXECUTION TIME: {} s'.format(time.time()-start))
except:
    print ('ERROR')

STARTING
set(['<http://www.wikidata.org/entity/Q4167410>', '<http://www.wikidata.org/entity/Q5>', '<http://www.wikidata.org/entity/Q13406463>', '<http://www.wikidata.org/entity/Q532>', '<http://www.wikidata.org/entity/Q79007>', '<http://www.wikidata.org/entity/Q16521>', '<http://www.wikidata.org/entity/Q13100073>', '<http://www.wikidata.org/entity/Q486972>', '<http://www.wikidata.org/entity/Q4167836>', '<http://www.wikidata.org/entity/Q11266439>'])
DONE
INDEGREE RESULT:
[('<http://www.wikidata.org/entity/Q5>', 265), ('<http://www.wikidata.org/entity/Q16521>', 44), ('<http://www.wikidata.org/entity/Q4167836>', 22), ('<http://www.wikidata.org/entity/Q4167410>', 12), ('<http://www.wikidata.org/entity/Q13406463>', 10), ('<http://www.wikidata.org/entity/Q532>', 8), ('<http://www.wikidata.org/entity/Q79007>', 4)]
OUTDEGREE RESULT
[]
EXECUTION TIME: 7.01688504219 s


In [6]:
# class instances indegree & outdegree

import os
import time
import string
import itertools
import re
import operator
import numpy

start = time.time()

class ClassInstances:
    def __init__(self, name, uri):
        self.name = name
        self.uri = uri
        self.allClassInstances = set()
        self.countDict = dict()
        self.min = 9999999
        self.max = 0
        self.avg = 0.0
        self.median = 0.0
    def getURI(self):
        return self.uri
    def getClassInstances(self):
        return len(self.allClassInstances)
    def addInstance(self, instance):
        if instance in self.allClassInstances:
            self.countDict[instance] += 1
        else:
            self.countDict[instance] = 1
            self.allClassInstances.add(instance)
    def calculateDegrees(self):
        allValueList = []
        for k,v in self.countDict.iteritems():
            allValueList.append(v)
            if (v < self.min):
                self.min = v
            if (v > self.max):
                self.max = v
            self.avg += v
        if (len(self.countDict) != 0):
            self.avg = self.avg / len(self.countDict)
        if (len(allValueList) != 0):
            self.median = numpy.median(numpy.array(allValueList))
    def printResults(self):
        print ('{}'.format(self.name))
        print ('min: {}, avg: {}, median: {}, max: {}'.format(self.min, self.avg, self.median, self.max))
        
# instances sets
iAllSet = set() #allInstances = set()
iSet1 = set() #instancesObject = set()
iSet2 = set() #instancesObjectNoOverlap = set()
iSet3 = set() #instancesNoun = set()
iSet4 = set() #instancesOrganism = set()
#iSet5 = set() #instancesIndividual = set()
#iSet6 = set() #instancesProgram = set()
#iSet7 = set() #instancesCollection = set()
#iSet8 = set() #instancesCity = set()
#iSet9 = set() #instancesTempObject = set()
#iSet10 = set() #instancesCoorp = set()
iSetAll = [iSet1, iSet2, iSet3, iSet4] #, iSet5, iSet6, iSet7, iSet8, iSet9, iSet10]

# indegree class instantiation 
ci1 = ClassInstances('indegree ', '<http://www.wikidata.org/prop/novalue/P37>')
ci2 = ClassInstances('indegree ', '<http://www.wikidata.org/prop/novalue/P102>')
ci3 = ClassInstances('indegree ', '<http://www.wikidata.org/prop/novalue/P155>')
ci4 = ClassInstances('indegree ', '<http://www.wikidata.org/prop/novalue/P2555>')
#ci5 = ClassInstances('indegree ', '')
#ci6 = ClassInstances('indegree ', '')
#ci7 = ClassInstances('indegree ', '')
#ci8 = ClassInstances('indegree ', '')
#ci9 = ClassInstances('indegree ', '')
#ci10 = ClassInstances('indegree ', '')
#ci10 = ClassInstances('indegree public coorporation', '<http://sw.opencyc.org/concept/Mx4rvVjZ_ZwpEbGdrcN5Y29ycA>')
ciAll = [ci1, ci2, ci3, ci4] #, ci5, ci6, ci7, ci8, ci9, ci10]

# outdegree class instantiation 
co1 = ClassInstances('outdegree ', '')
co2 = ClassInstances('outdegree ', '')
co3 = ClassInstances('outdegree ', '')
co4 = ClassInstances('outdegree ', '')
#co5 = ClassInstances('outdegree ', '')
#co6 = ClassInstances('outdegree ', '')
#co7 = ClassInstances('outdegree ', '')
#co8 = ClassInstances('outdegree ', '')
#co9 = ClassInstances('outdegree ', '')
#co10 = ClassInstances('outdegree ', '')
#co10 = ClassInstances('outdegree public coorporation', '<http://sw.opencyc.org/concept/Mx4rvVjZ_ZwpEbGdrcN5Y29ycA>')
coAll = [co1, co2, co3, co4] #, co5, co6, co7, co8, co9, co10]

def getSPO(splittedLine):
    word_position = 0
    for word in splittedLine:
        if (word_position == 0):
            subj = word
        elif (word_position == 1):
            pred = word
        elif (word_position == 2):
            obj = word
        else:
            return subj, pred, obj
        word_position += 1
    return subj, pred, obj

# count all instances of the top10 classes
def countInstances(s, o):
    #s a o: o=class s=instance
    iAllSet.add(s)
    if (o == ci1.getURI()):
        if (s not in iSet1):
            iSet1.add(s)
        return
    if (o == ci2.getURI()):
        if (s not in iSet2):
            iSet2.add(s)
        return
    if (o == ci3.getURI()):
        if (s not in iSet3):
            iSet3.add(s)
        return
    if (o == ci4.getURI()):
        if (s not in iSet4):
            iSet4.add(s)
        return
    if (o == ci5.getURI()):
        if (s not in iSet5):
            iSet5.add(s)
        return
    if (o == ci6.getURI()):
        if (s not in iSet6):
            iSet6.add(s)
        return
    if (o == ci7.getURI()):
        if (s not in iSet7):
            iSet7.add(s)
        return
    if (o == ci8.getURI()):
        if (s not in iSet8):
            iSet8.add(s)
        return
    if (o == ci9.getURI()):
        if (s not in iSet9):
            iSet9.add(s)
        return
    if (o == ci10.getURI()):
        if (s not in iSet10):
            iSet10.add(s)
        return

# count instance degrees
def countInstanceDegrees(s, o):
    if (s in iAllSet): # outdegree
        if (s in iSet1):
            co1.addInstance(s)
        elif (s in iSet2):
            co2.addInstance(s)
        elif (s in iSet3):
            co3.addInstance(s)
        elif (s in iSet4):
            co4.addInstance(s)
        elif (s in iSet5):
            co5.addInstance(s)
        elif (s in iSet6):
            co6.addInstance(s)
        elif (s in iSet7):
            co7.addInstance(s)
        elif (s in iSet8):
            co8.addInstance(s)
        elif (s in iSet9):
            co9.addInstance(s)
        elif (s in iSet10):
            co10.addInstance(s)       
    if (o in iAllSet): # indegree
        if (o in iSet1):
            ci1.addInstance(o)
        elif (o in iSet2):
            ci2.addInstance(o)
        elif (o in iSet3):
            ci3.addInstance(o)
        elif (o in iSet4):
            ci4.addInstance(o)
        elif (o in iSet5):
            ci5.addInstance(o)
        elif (o in iSet6):
            ci6.addInstance(o)
        elif (o in iSet7):
            ci7.addInstance(o)
        elif (o in iSet8):
            ci8.addInstance(o)
        elif (o in iSet9):
            ci9.addInstance(o)
        elif (o in iSet10):
            ci10.addInstance(o)
        return

def calculateClassDegrees():
    for item in ciAll:
        item.calculateDegrees()
    for item in coAll:
        item.calculateDegrees()
        
def printClassDegreeResults():
    for item in ciAll:
        item.printResults()
    for item in coAll:
        item.printResults()
        
print('START')
try:
    # get all instances for the top10 classes
    f = open(readFile, 'r')
    lineCounter = 0
    print ('START COUNTING INSTANCES')
    for line in f:
        splittedLine = line.rstrip('\n').split()
        s, p, o = getSPO(splittedLine)
        if (p == rdfType and top10dic.has_key(o)):
            countInstances(s, o)
        lineCounter += 1
        if (lineCounter % lineProgress == 0):
            print ('{} lines read'.format(lineCounter))
        #if (lineCounter > 5000):
        #    break
    f.close()
    print ('DONE COUNTING INSTANCES')
    for i, item in enumerate(iSetAll):
        print ('iSet{}, #instances: {}'.format(i+1, len(iSetAll[i])))
        
    # count instance degrees
    print ('START COUNTING INSTANCE DEGREES')
    f = open(readFile, 'r')
    lineCounter = 0
    for line in f:
        splittedLine = line.rstrip('\n').split()
        s, p, o = getSPO(splittedLine)
        countInstanceDegrees(s, o)
        lineCounter += 1
        if (lineCounter % lineProgress == 0):
            print ('{} lines read'.format(lineCounter))
        #if (lineCounter > 5000):
        #    break
    f.close()
    print ('DONE COUNTING INSTANCE DEGREES')
    print ('START CALCULATING CLASS DEGREES')
    calculateClassDegrees()
    print ('DONE CALCULATING CLASS DEGREES')
    print ('RESULTS')
    printClassDegreeResults()   
    print ('EXECUTION TIME: {} s'.format(time.time()-start))
except:
    print ('ERROR')

START
START COUNTING INSTANCES
DONE COUNTING INSTANCES
iSet1, #instances: 8
iSet2, #instances: 4
iSet3, #instances: 1
iSet4, #instances: 1
START COUNTING INSTANCE DEGREES
DONE COUNTING INSTANCE DEGREES
START CALCULATING CLASS DEGREES
DONE CALCULATING CLASS DEGREES
RESULTS
indegree 
min: 1, avg: 92.25, median: 92.0, max: 184
indegree 
min: 1, avg: 36.25, median: 31.0, max: 82
indegree 
min: 1, avg: 1.0, median: 1.0, max: 1
indegree 
min: 1, avg: 1.0, median: 1.0, max: 1
outdegree 
min: 4, avg: 336.0, median: 316.0, max: 698
outdegree 
min: 5, avg: 177.75, median: 175.5, max: 355
outdegree 
min: 6, avg: 6.0, median: 6.0, max: 6
outdegree 
min: 5, avg: 5.0, median: 5.0, max: 5
EXECUTION TIME: 7.04606103897 s
